# lr-tfidf데모

# 라이브러리 import 및 설정

In [42]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
import warnings
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [43]:
data_dir    = Path('C:\\Users\\USER\\Desktop\\open\\')
tst_dir     = Path('C:\\Users\\USER\\Desktop\\open\\')
feature_dir = Path('C:\\Users\\USER\\Desktop\\open\\feature\\')
sub_dir     = Path('C:\\Users\\USER\\Desktop\\open\\sub\\')
val_dir     = Path('C:\\Users\\USER\\Desktop\\open\\val\\')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

algo_name = 'lr'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

## train 데이터 살펴보기

In [44]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(54879, 2)


,text,author
index,,
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


## test 데이터 살펴보기

In [45]:
tst = pd.read_csv(tst_file, index_col=0)
print(tst.shape)
tst.head()

(19617, 1)


,text
index,
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


## NLTK 예시_전처리
* 자연어 처리(NLP) 를 위한 자연어 라이브러리(nltk,spacy) 중 하나
* 피처 가공에 따라 라이브러리 달리할 것.
* EX> spacy에서 전처리와 피처가공 툴을 같이 제공하기 때문에 추가로 피처가공 틀인 sklearn or keras 필요X
* EX> NLTK 로 전처리과정을 거치고 다른 피처 가공 툴 사용도 가능 
* 피처가공처리의 필요도가 낮은 알고리즘, 신경망 사용으로 자연어처리할 경우 가공단계 필요 X 
* Tokenizer, Lemmatization, Stemming..

In [46]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
s = trn.text[4]
print(s)

“Have mercy, gentlemen!” odin flung up his hands. “Don’t write that, anyway; have some shame. Here I’ve torn my heart asunder before you, and you seize the opportunity and are fingering the wounds in both halves.... Oh, my God!”


In [47]:
tokens = word_tokenize(s)
print(tokens)

['“', 'Have', 'mercy', ',', 'gentlemen', '!', '”', 'odin', 'flung', 'up', 'his', 'hands', '.', '“', 'Don', '’', 't', 'write', 'that', ',', 'anyway', ';', 'have', 'some', 'shame', '.', 'Here', 'I', '’', 've', 'torn', 'my', 'heart', 'asunder', 'before', 'you', ',', 'and', 'you', 'seize', 'the', 'opportunity', 'and', 'are', 'fingering', 'the', 'wounds', 'in', 'both', 'halves', '....', 'Oh', ',', 'my', 'God', '!', '”']


In [48]:
#언어학적으로 상세한 규칙을 적용하여 변환
#대소문자는 그대로, 단복수는 복수로 변환 
lemmatizer = WordNetLemmatizer()
[lemmatizer.lemmatize(t) for t in tokens]

['“',
 'Have',
 'mercy',
 ',',
 'gentleman',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'Don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'Here',
 'I',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asunder',
 'before',
 'you',
 ',',
 'and',
 'you',
 'seize',
 'the',
 'opportunity',
 'and',
 'are',
 'fingering',
 'the',
 'wound',
 'in',
 'both',
 'half',
 '....',
 'Oh',
 ',',
 'my',
 'God',
 '!',
 '”']

In [49]:
#위보다 빠르고 성능도 비슷
stemmer = SnowballStemmer("english")
[stemmer.stem(t) for t in tokens]

['“',
 'have',
 'merci',
 ',',
 'gentlemen',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'here',
 'i',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asund',
 'befor',
 'you',
 ',',
 'and',
 'you',
 'seiz',
 'the',
 'opportun',
 'and',
 'are',
 'finger',
 'the',
 'wound',
 'in',
 'both',
 'halv',
 '....',
 'oh',
 ',',
 'my',
 'god',
 '!',
 '”']

[문자열 가공]  
1. Bag-of-Words(simple):countVectorizer, TfidVectorizer  
2. Hashing Trick: hashingVectorizer  
3. Embeddings(with 신경망): Word2Vec, GloVe

## Bag-of-Words 피처 생성

In [81]:
#각 단어의 빈도를 피처로 사용
#생성 피처 조절(필수):mon_df,max_feature
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
X_tst = vec.fit_transform(tst['text'])
print(X_cnt.shape,X_tst.shape)#열 크기 같게 만들어줘야함 

(54879, 2685) (19617, 1920)


In [68]:
#0에 집중되어 있는 멱함수 분포를 뜀
#->로지스틱 회귀 도는 뉴럴 네트워크 알고리즘과 같이 사용하여 정규화 과정이 필요
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [72]:
#정규화 또는 표준화 과정없이 피처 생성 툴
#각단어의 빈도를 역문서 빈도로 나눈 값을 피처로 사용
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)
#학습에 적용할때
#가공한 X,X_tst 변수의 경우 형태가 numpy와 비슷하여 sklearn에 바로 적용가능+lgbm, xgboost ok
#그러나 파이토치(pytorch),텐서플로,keras, 뉴럴Network 신경망 적용시 "".todense()"" 함수로 변환과정 필요

(54879, 5897) (19617, 5897)


In [73]:
X[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

## Hashing Trick 피처 생성

In [56]:
#Hashing Trick:각각 범주를 Hash함수에 적용한 값으로 변환
#->fit() 필요X fast low memory
vectorize = HashingVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3),
    n_features=7               # 기본 feature 수를 설정하며 기본값이 2의 20승이다. 아래 예시를 위해 feature 를 7로 한정했으나, 아래 유사문장을 찾을때는 다시 n_features 주석처리 했다.
)
X_hv = vectorize.fit_transform(trn['text'])
print(X_hv.shape)
print(X_hv.toarray())#todense()처럼 변환하여 0과 값을 같이 보여줌

(54879, 7)
[[-0.6644106   0.5694948   0.1898316  ... -0.0949158  -0.1898316
  -0.3796632 ]
 [-0.19611614 -0.78446454 -0.39223227 ... -0.19611614  0.
   0.        ]
 [-0.1933473  -0.67671554  0.3866946  ...  0.58004189 -0.09667365
   0.        ]
 ...
 [-0.56011203 -0.28005602  0.56011203 ...  0.28005602 -0.14002801
   0.14002801]
 [-0.24253563 -0.24253563 -0.24253563 ...  0.          0.72760688
  -0.24253563]
 [ 0.34299717 -0.51449576 -0.51449576 ... -0.17149859  0.17149859
  -0.51449576]]


In [57]:
# 추가로 back of word or hashing 의 경우 문자열 길이 통합을 위한 padding 작업X
# 문자열 입력을 자체로 받는 경우 padding 요구 EX> 뉴럴네트워크 에서의 문자모델링에서 요구함

## Stratified K-Fold Cross Validation
*Stratified N-Fold CV: N-Fold CV에서 각각의 폴드에서 종속변수의 분포가 동일하도록 폴드를 나누는 방식.
현재 사용하는 데이터처럼 분류학습에서 종속변수의 범주의 분포가 균일하지 않을 때 사용된다.

In [74]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

## 로지스틱회귀 모델 학습

In [78]:
y = trn.author.values
y.shape
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

In [79]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')#리더보드_0.5052060884

Accuracy (CV):  76.6140%
Log Loss (CV):   0.6800


In [77]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

## 제출 파일 생성

In [62]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [63]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0631,0.5302,0.3155,0.0659,0.0253
1,0.0815,0.8202,0.0032,0.0269,0.0682
2,0.7208,0.0319,0.1174,0.0381,0.0918
3,0.0392,0.0036,0.8465,0.0058,0.1049
4,0.3044,0.2440,0.1450,0.1905,0.1161


In [64]:
sub.to_csv(sub_file)